In [1]:
# # Cell 1: Installations -> Used venv instead
!pip install transformers torch accelerate bitsandbytes
!pip install scikit-learn pandas
!pip install sentence-transformers
!pip install thefuzz[speedup]
!pip install tqdm # For progress bars

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 48.7 MB/s eta 0:00:00


In [2]:
# Cell 3: Imports and Path Configuration
import os
import re
import random
import pandas as pd
from tqdm.auto import tqdm
from collections import defaultdict

# For LLM and Embeddings
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer, util

# For String Matching
from thefuzz import process as fuzzy_process

# For Evaluation
from sklearn.metrics import precision_score, recall_score, f1_score

DATA_DIR = '/content/drive/MyDrive/cadec'  # Path to the CADEC dataset
# ---

# Defining paths to sub-directories
TEXT_DIR = os.path.join(DATA_DIR, 'text')
ORIGINAL_DIR = os.path.join(DATA_DIR, 'original')
MEDDRA_DIR = os.path.join(DATA_DIR, 'meddra')
SCT_DIR = os.path.join(DATA_DIR, 'sct')

print(f"Data directory set to: {os.path.abspath(DATA_DIR)}")

Data directory set to: /content/drive/MyDrive/cadec


In [3]:
# Cell 4: Task 1 - Enumerate Distinct Entities
def enumerate_entities():
    """
    Parses all files in the 'original' directory to find distinct entities
    for each label type.
    """
    # A dictionary to hold a set of entities for each label
    distinct_entities = defaultdict(set)

    # Regex to parse a line in the 'original' files
    # Captures: T-ID, Label, and Text
    line_regex = re.compile(r'^(T\d+)\t(\w+)[\s\d;]+\t(.+)$')

    print(f"Processing files in: {ORIGINAL_DIR}")
    filenames = os.listdir(ORIGINAL_DIR)

    for filename in tqdm(filenames, desc="Parsing Original Files"):
        if not filename.endswith('.ann'):
            continue

        filepath = os.path.join(ORIGINAL_DIR, filename)
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                # Ignore comment lines
                if line.startswith('#'):
                    continue

                match = line_regex.match(line.strip())
                if match:
                    _, label, text = match.groups()
                    # Normalize text for accurate counting (lowercase, strip whitespace)
                    normalized_text = text.strip().lower()
                    if normalized_text:
                        distinct_entities[label].add(normalized_text)

    return distinct_entities

# Run the function and print the results
distinct_entities_by_label = enumerate_entities()

print("\n--- Distinct Entity Counts ---")
for label, entities in distinct_entities_by_label.items():
    print(f"Label: {label}, Count: {len(entities)}")
    # Print a few examples
    print(f"  Examples: {list(entities)[:3]}\n")

Processing files in: /content/drive/MyDrive/cadec/original


Parsing Original Files:   0%|          | 0/1250 [00:00<?, ?it/s]


--- Distinct Entity Counts ---
Label: ADR, Count: 3400
  Examples: ['cant sleep', 'blood clots in urine', 'numbness in left toe']

Label: Drug, Count: 323
  Examples: ['vitorin', 'niaspan', 'magnesium orotate']

Label: Finding, Count: 298
  Examples: ['als', 'visual anomalies', 'shingles']

Label: Disease, Count: 164
  Examples: ['als', 'heel spurs', 'ibs']

Label: Symptom, Count: 148
  Examples: ['sore knee', 'constant pain', 'chronic bone/joint pain']



In [4]:
!pip install python-dotenv
!pip install -U bitsandbytes

In [5]:
# Cell 5: Setup LLM for Task 2
# Define model and quantization configuration
from google.colab import userdata
hf_token=userdata.get('hf_token')

if not hf_token:
     raise ValueError("Hugging Face token not found.")

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

print("Loading tokenizer and model...")
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    token=hf_token
)
# Add a padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

print("Model loaded successfully!")


Loading tokenizer and model...


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Model loaded successfully!


In [14]:

def generate_bio_labels(forum_post_text: str) -> str:
    prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a medical NER tagger. Tag EVERY SINGLE WORD in the exact format shown.

RULES:
1. Tag EVERY word (no skipping)
2. Format: word/TAG word/TAG word/TAG
3. NO other text - ONLY tagged words
4. Start entities with B-, continue with I-
5. Numbers = O (not Drug)

TAGS: B-ADR, I-ADR, B-Drug, I-Drug, B-Disease, I-Disease, B-Symptom, I-Symptom, O<|eot_id|><|start_header_id|>user<|end_header_id|>

EXAMPLE 1:
Input: I feel a bit drowsy
Output: I/O feel/B-ADR a/I-ADR bit/I-ADR drowsy/I-ADR

EXAMPLE 2:
Input: no gastric problems after Lipitor
Output: no/O gastric/B-ADR problems/I-ADR after/O Lipitor/B-Drug

EXAMPLE 3:
Input: arthritis pain got worse
Output: arthritis/B-Disease pain/B-Symptom got/O worse/O

Now tag this text (EVERY word, NO explanations):
{text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""

    prompt = prompt_template.format(text=forum_post_text)

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=1200
    ).to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=3072,
        do_sample=True,
        temperature=0.05,  # Very low
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=False)

    if "<|start_header_id|>assistant<|end_header_id|>" in response:
        output = response.split("<|start_header_id|>assistant<|end_header_id|>")[-1]
        output = output.replace("<|eot_id|>", "").replace("<|end_of_text|>", "").strip()
    else:
        output = response.strip()

    return output

# Test
sample_filename = 'ARTHROTEC.1.txt'
sample_filepath = os.path.join(TEXT_DIR, sample_filename)

with open(sample_filepath, 'r', encoding='utf-8') as f:
    sample_text = f.read()

print("--- Generating BIO labels with Llama-3-8B ---")
print(f"Input: {sample_text[:150]}...\n")

bio_labeled_output = generate_bio_labels(sample_text)

print(f"=== LLM OUTPUT ({len(bio_labeled_output)} chars) ===")
print(bio_labeled_output[:600])

--- Generating BIO labels with Llama-3-8B ---
Input: I feel a bit drowsy & have a little blurred vision, so far no gastric problems.
I've been on Arthrotec 50 for over 10 years on and off, only taking it...

=== LLM OUTPUT (708 chars) ===
I/O feel/B-ADR a/I-AD &/O have/O a/I-ADR little/I-ADR blurred/I-AD vision/I-AD, so/O far/O no/O gastric/B-ADR problems/I-AD.
I've/O been/O on/O Arthrotec/B-Drug 50/O for/O over/O 10/O years/O on/O &/O off/O, only/O taking/O it/O when/O I/O needed/O it/O.
Due/O to/O my/B-Disease getting/O progressively/O worse/O, to/O the/O point/O where/O I/O am/O in/O tears/O with/O the/O agony/O, gp's/O started/O me/O on/O 75/O twice/O a/O day/O and/O I/O have/O to/O take/O it/O.
every/O day/O for/O the/O next/O month/O to/O see/O how/O I/O get/O on/O, here/O goes/O.
So/O far/O it/O has/O been/O very/O good/


In [26]:
def bio_to_original_format(bio_string: str, original_text: str):
    """
    Convert BIO with AGGRESSIVE error handling.
    """
    if '/' not in bio_string:
        print("ERROR: No valid BIO tags found")
        return []

    VALID_TAGS = {'B-ADR', 'I-ADR', 'B-DRUG', 'I-DRUG',
                  'B-DISEASE', 'I-DISEASE', 'B-SYMPTOM', 'I-SYMPTOM', 'O'}

    entities = []
    current_entity = None
    entity_words = []

    tokens = bio_string.split()

    for token in tokens:
        # Strip ALL trailing punctuation from token
        token = token.rstrip('.,;:!?/').strip()

        if not token or '/' not in token:
            continue

        # Handle malformed tokens like "/S-ADR"
        if token.startswith('/'):
            continue

        parts = token.rsplit('/', 1)
        if len(parts) != 2:
            continue

        word, tag = parts

        # Clean word and tag
        word = word.strip('.,;:!?/')
        tag = tag.strip('.,;:!?/').upper()

        if not word or word in ['/', ',', '.']:
            continue

        # Normalize tag
        tag = tag.replace('DRUG', 'DRUG').replace('SYMPTOM', 'SYMPTOM')

        # STRICT: Only accept valid tags
        if tag not in VALID_TAGS:
            # Silently skip - don't spam warnings
            if current_entity and entity_words:
                entities.append((current_entity, entity_words))
            current_entity = None
            entity_words = []
            continue

        # Process valid tags
        if tag.startswith('B-'):
            if current_entity and entity_words:
                entities.append((current_entity, entity_words))
            current_entity = tag[2:]
            entity_words = [word]

        elif tag.startswith('I-'):
            label = tag[2:]
            if current_entity == label:
                entity_words.append(word)
            else:
                if current_entity and entity_words:
                    entities.append((current_entity, entity_words))
                current_entity = label
                entity_words = [word]

        elif tag == 'O':
            if current_entity and entity_words:
                entities.append((current_entity, entity_words))
            current_entity = None
            entity_words = []

    if current_entity and entity_words:
        entities.append((current_entity, entity_words))

    if len(entities) == 0:
        print("WARNING: No valid entities extracted from BIO output")
        return []

    # Rest of mapping logic stays the same...
    final_entities = []
    original_lower = original_text.lower()

    for label, words in entities:
        entity_text = ' '.join(words)

        if len(entity_text) <= 1 or (entity_text.isdigit() and len(entity_text) <= 2):
            continue

        entity_lower = entity_text.lower()
        start_idx = original_lower.find(entity_lower)

        if start_idx != -1:
            actual_text = original_text[start_idx:start_idx + len(entity_text)]
            final_entities.append({
                'label': label,
                'text': actual_text,
                'start': start_idx,
                'end': start_idx + len(entity_text)
            })
        else:
            pattern = r'\s+'.join(re.escape(w) for w in words)
            match = re.search(pattern, original_lower, re.IGNORECASE)
            if match:
                actual_text = original_text[match.start():match.end()]
                final_entities.append({
                    'label': label,
                    'text': actual_text,
                    'start': match.start(),
                    'end': match.end()
                })

    # Context-aware ADR detection
    for entity in final_entities:
        if entity['label'] == 'SYMPTOM':
            context_start = max(0, entity['start'] - 120)
            context_end = min(len(original_text), entity['end'] + 120)
            context = original_text[context_start:context_end].lower()

            drug_phrases = ['taking', 'take it', 'on', 'after', 'when on', 'started me on', 'caused']
            if any(phrase in context for phrase in drug_phrases):
                has_drug = any(e['label'] == 'DRUG' and abs(e['start'] - entity['start']) < 150
                              for e in final_entities)
                if has_drug:
                    entity['label'] = 'ADR'

    seen = set()
    unique = []
    for ent in final_entities:
        key = (ent['label'], ent['text'].lower().strip())
        if key not in seen:
            seen.add(key)
            unique.append(ent)

    unique.sort(key=lambda x: x['start'])
    return unique

print("\n--- Converting to entities ---")
llm_predictions = bio_to_original_format(bio_labeled_output, sample_text)

print("\n=== EXTRACTED ENTITIES ===")
for entity in llm_predictions:
    print(f"{entity['label']:10} | {entity['start']:3}-{entity['end']:3} | {entity['text']}")


--- Converting to entities ---

=== EXTRACTED ENTITIES ===
ADR        |   2-  6 | feel
ADR        |  27- 35 | a little
ADR        |  62- 69 | gastric
DRUG       |  93-102 | Arthrotec
DISEASE    | 176-178 | my


In [27]:
# Cell 8: Improved matching with partial overlap tolerance
def parse_ground_truth(filepath: str):
    """Parse ground truth from .ann file"""
    entities = []
    line_regex = re.compile(r'^(T\d+)\t(\w+)\s([\d\s;]+)\t(.+)$')

    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            if line.startswith('#'):
                continue
            match = line_regex.match(line.strip())
            if match:
                _, label, _, text = match.groups()
                entities.append({
                    'label': label,
                    'text': text.strip()
                })
    return entities

def calculate_performance(predictions, ground_truth):
    """
    Calculate P/R/F1 with relaxed word overlap matching.
    Matches if: same label + >66% word overlap (looser than 50%)
    """
    tp = 0
    matched_gt_indices = set()

    for pred in predictions:
        pred_text = pred['text'].lower().strip()
        pred_words = set(pred_text.split())

        best_match = None
        best_similarity = 0

        for idx, gt in enumerate(ground_truth):
            if idx in matched_gt_indices:
                continue

            # Label must match
            if pred['label'].upper() != gt['label'].upper():
                continue

            gt_text = gt['text'].lower().strip()
            gt_words = set(gt_text.split())

            # Jaccard similarity
            if pred_words and gt_words:
                intersection = len(pred_words & gt_words)
                union = len(pred_words | gt_words)
                similarity = intersection / union if union > 0 else 0

                if similarity > best_similarity:
                    best_similarity = similarity
                    best_match = idx

        # Match if similarity > 0.66 (relaxed from 0.5)
        if best_match is not None and best_similarity > 0.66:
            tp += 1
            matched_gt_indices.add(best_match)

    fp = len(predictions) - tp
    fn = len(ground_truth) - len(matched_gt_indices)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'tp': tp,
        'fp': fp,
        'fn': fn
    }

# Load and evaluate
sample_ann_filename = sample_filename.replace('.txt', '.ann')
ground_truth_path = os.path.join(ORIGINAL_DIR, sample_ann_filename)
ground_truth_entities = parse_ground_truth(ground_truth_path)

print(f"\n=== GROUND TRUTH ({len(ground_truth_entities)} entities) ===")
for gt in ground_truth_entities:
    print(f"{gt['label']:10} | {gt['text']}")

performance = calculate_performance(llm_predictions, ground_truth_entities)

print(f"\n=== PERFORMANCE FOR {sample_filename} ===")
print(f"Precision: {performance['precision']:.4f}")
print(f"Recall:    {performance['recall']:.4f}")
print(f"F1-Score:  {performance['f1']:.4f}")
print(f"\nTP: {performance['tp']}, FP: {performance['fp']}, FN: {performance['fn']}")

# Detailed matching analysis
print("\n=== MATCHING DETAILS ===")
matched_gt = set()
for pred in llm_predictions:
    pred_words = set(pred['text'].lower().split())

    for idx, gt in enumerate(ground_truth_entities):
        if pred['label'].upper() == gt['label'].upper():
            gt_words = set(gt['text'].lower().split())
            if pred_words and gt_words:
                intersection = len(pred_words & gt_words)
                union = len(pred_words | gt_words)
                similarity = intersection / union if union > 0 else 0

                if similarity > 0.66:
                    print(f"✓ MATCH: '{pred['text']}' ↔ '{gt['text']}' (sim={similarity:.2f})")
                    matched_gt.add(idx)
                    break
    else:
        print(f"✗ MISS:  '{pred['text']}' ({pred['label']})")

print("\nUnmatched ground truth:")
for idx, gt in enumerate(ground_truth_entities):
    if idx not in matched_gt:
        print(f"✗ '{gt['text']}' ({gt['label']})")


=== GROUND TRUTH (8 entities) ===
ADR        | bit drowsy
ADR        | little blurred vision
Drug       | Arthrotec
Disease    | arthritis
Symptom    | agony
ADR        | gastric problems
Symptom    | pains
ADR        | feel a bit weird

=== PERFORMANCE FOR ARTHROTEC.1.txt ===
Precision: 0.2000
Recall:    0.1250
F1-Score:  0.1538

TP: 1, FP: 4, FN: 7

=== MATCHING DETAILS ===
✗ MISS:  'feel' (ADR)
✗ MISS:  'a little' (ADR)
✗ MISS:  'gastric' (ADR)
✓ MATCH: 'Arthrotec' ↔ 'Arthrotec' (sim=1.00)
✗ MISS:  'my' (DISEASE)

Unmatched ground truth:
✗ 'bit drowsy' (ADR)
✗ 'little blurred vision' (ADR)
✗ 'arthritis' (Disease)
✗ 'agony' (Symptom)
✗ 'gastric problems' (ADR)
✗ 'pains' (Symptom)
✗ 'feel a bit weird' (ADR)


In [28]:
# Cell 9: Task 4 - ADR Performance against MedDRA (with debugging)
def parse_meddra_truth(filepath: str):
    """Parse MedDRA ground truth file"""
    entities = []
    line_regex = re.compile(r'^(TT\d+)\t\d+\s[\d\s]+\t(.+)$')

    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            match = line_regex.match(line.strip())
            if match:
                _, text = match.groups()
                entities.append({'label': 'ADR', 'text': text.strip()})
    return entities

# Filter predictions for ADR only
llm_adr_predictions = [e for e in llm_predictions if e['label'] == 'ADR']

# Load MedDRA ground truth
meddra_ann_filename = sample_filename.replace('.txt', '.ann')
meddra_ground_truth = parse_meddra_truth(os.path.join(MEDDRA_DIR, meddra_ann_filename))

print(f"\n=== ADR COMPARISON ===")
print(f"\nPREDICTED ADRs ({len(llm_adr_predictions)}):")
for pred in llm_adr_predictions:
    print(f"  - '{pred['text']}'")

print(f"\nMEDDRA GROUND TRUTH ({len(meddra_ground_truth)}):")
for gt in meddra_ground_truth:
    print(f"  - '{gt['text']}'")

# Calculate and show matching details
adr_performance = calculate_performance(llm_adr_predictions, meddra_ground_truth)

print(f"\n=== MATCHING ANALYSIS ===")
for pred in llm_adr_predictions:
    pred_words = set(pred['text'].lower().split())

    best_match = None
    best_sim = 0

    for gt in meddra_ground_truth:
        gt_words = set(gt['text'].lower().split())
        if pred_words and gt_words:
            intersection = len(pred_words & gt_words)
            union = len(pred_words | gt_words)
            sim = intersection / union if union > 0 else 0
            if sim > best_sim:
                best_sim = sim
                best_match = gt['text']

    if best_sim > 0.66:
        print(f"✓ '{pred['text']}' → '{best_match}' (sim={best_sim:.2f})")
    else:
        print(f"✗ '{pred['text']}' → best='{best_match}' (sim={best_sim:.2f}) FAILED")

print(f"\n=== ADR-ONLY PERFORMANCE (vs MedDRA) ===")
print(f"Precision: {adr_performance['precision']:.4f}")
print(f"Recall:    {adr_performance['recall']:.4f}")
print(f"F1-Score:  {adr_performance['f1']:.4f}")


=== ADR COMPARISON ===

PREDICTED ADRs (3):
  - 'feel'
  - 'a little'
  - 'gastric'

MEDDRA GROUND TRUTH (4):
  - 'bit drowsy'
  - 'little blurred vision'
  - 'gastric problems'
  - 'feel a bit weird'

=== MATCHING ANALYSIS ===
✗ 'feel' → best='feel a bit weird' (sim=0.25) FAILED
✗ 'a little' → best='little blurred vision' (sim=0.25) FAILED
✗ 'gastric' → best='gastric problems' (sim=0.50) FAILED

=== ADR-ONLY PERFORMANCE (vs MedDRA) ===
Precision: 0.0000
Recall:    0.0000
F1-Score:  0.0000


In [29]:
# Cell 10: Task 5 - Evaluate Random Files with Detailed Logging
def evaluate_single_file(filename: str, verbose=False):
    """Complete evaluation pipeline for one file with optional verbose output"""
    try:
        # Read text
        with open(os.path.join(TEXT_DIR, filename), 'r', encoding='utf-8') as f:
            text = f.read()

        if verbose:
            print(f"\n{'='*70}")
            print(f"FILE: {filename}")
            print(f"{'='*70}")
            print(f"INPUT TEXT ({len(text)} chars):")
            print(text[:300] + "..." if len(text) > 300 else text)
            print(f"\n{'---'*23}")

        # Generate predictions
        bio_output = generate_bio_labels(text)

        if verbose:
            print(f"\nLLM BIO OUTPUT ({len(bio_output)} chars):")
            # Show first 500 chars of BIO output
            print(bio_output[:500] + "..." if len(bio_output) > 500 else bio_output)
            print(f"\n{'---'*23}")

        predictions = bio_to_original_format(bio_output, text)

        if verbose:
            print(f"\nEXTRACTED ENTITIES ({len(predictions)}):")
            if predictions:
                for pred in predictions:
                    print(f"  {pred['label']:10} | {pred['start']:3}-{pred['end']:3} | {pred['text']}")
            else:
                print("  (none)")
            print(f"\n{'---'*23}")

        # Load ground truth
        ann_filename = filename.replace('.txt', '.ann')
        ground_truth = parse_ground_truth(os.path.join(ORIGINAL_DIR, ann_filename))

        if verbose:
            print(f"\nGROUND TRUTH ({len(ground_truth)} entities):")
            for gt in ground_truth:
                print(f"  {gt['label']:10} | {gt['text']}")
            print(f"\n{'---'*23}")

        # Calculate performance
        performance = calculate_performance(predictions, ground_truth)

        if verbose:
            print(f"\nPERFORMANCE:")
            print(f"  Precision: {performance['precision']:.4f}")
            print(f"  Recall:    {performance['recall']:.4f}")
            print(f"  F1-Score:  {performance['f1']:.4f}")
            print(f"  TP: {performance['tp']}, FP: {performance['fp']}, FN: {performance['fn']}")

        return performance

    except Exception as e:
        print(f"ERROR in {filename}: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# Get random sample
all_files = [f for f in os.listdir(TEXT_DIR) if f.endswith('.txt')]
random.seed(42)  # For reproducibility
random_files = random.sample(all_files, min(50, len(all_files)))

print(f"\n{'='*70}")
print(f"EVALUATING {len(random_files)} RANDOM FILES")
print(f"{'='*70}")

# First, evaluate with verbose output for first 2 files
print("\n" + "="*70)
print("DETAILED OUTPUT FOR FIRST 2 FILES")
print("="*70)

all_scores = []
for i, filename in enumerate(random_files[:2]):
    scores = evaluate_single_file(filename, verbose=True)
    if scores:
        all_scores.append(scores)

# Then evaluate remaining files without verbose output
print("\n" + "="*70)
print(f"PROCESSING REMAINING {len(random_files)-2} FILES (summary only)")
print("="*70)

for filename in tqdm(random_files[2:], desc="Processing"):
    scores = evaluate_single_file(filename, verbose=False)
    if scores:
        all_scores.append(scores)
        # Print one-line summary
        print(f"{filename:30} | P:{scores['precision']:.3f} R:{scores['recall']:.3f} F1:{scores['f1']:.3f}")

# Calculate macro-averaged metrics
if all_scores:
    avg_precision = sum(s['precision'] for s in all_scores) / len(all_scores)
    avg_recall = sum(s['recall'] for s in all_scores) / len(all_scores)
    avg_f1 = sum(s['f1'] for s in all_scores) / len(all_scores)

    print(f"\n{'='*70}")
    print(f"FINAL RESULTS ({len(all_scores)} files successfully processed)")
    print(f"{'='*70}")
    print(f"Macro-Avg Precision: {avg_precision:.4f}")
    print(f"Macro-Avg Recall:    {avg_recall:.4f}")
    print(f"Macro-Avg F1-Score:  {avg_f1:.4f}")

    # Distribution statistics
    f1_scores = [s['f1'] for s in all_scores]
    precision_scores = [s['precision'] for s in all_scores]
    recall_scores = [s['recall'] for s in all_scores]

    print(f"\nF1 Score Distribution:")
    print(f"  Min:    {min(f1_scores):.4f}")
    print(f"  Q1:     {sorted(f1_scores)[len(f1_scores)//4]:.4f}")
    print(f"  Median: {sorted(f1_scores)[len(f1_scores)//2]:.4f}")
    print(f"  Q3:     {sorted(f1_scores)[3*len(f1_scores)//4]:.4f}")
    print(f"  Max:    {max(f1_scores):.4f}")

    # Count failure cases
    zero_f1_count = sum(1 for s in f1_scores if s == 0.0)
    low_f1_count = sum(1 for s in f1_scores if 0 < s < 0.3)
    mid_f1_count = sum(1 for s in f1_scores if 0.3 <= s < 0.6)
    high_f1_count = sum(1 for s in f1_scores if s >= 0.6)

    print(f"\nPerformance Breakdown:")
    print(f"  Complete failures (F1=0):   {zero_f1_count}/{len(all_scores)} ({zero_f1_count/len(all_scores)*100:.1f}%)")
    print(f"  Poor (0 < F1 < 0.3):         {low_f1_count}/{len(all_scores)} ({low_f1_count/len(all_scores)*100:.1f}%)")
    print(f"  Moderate (0.3 ≤ F1 < 0.6):   {mid_f1_count}/{len(all_scores)} ({mid_f1_count/len(all_scores)*100:.1f}%)")
    print(f"  Good (F1 ≥ 0.6):             {high_f1_count}/{len(all_scores)} ({high_f1_count/len(all_scores)*100:.1f}%)")

    print(f"\n{'='*70}")

else:
    print("\nERROR: No files successfully processed!")


EVALUATING 50 RANDOM FILES

DETAILED OUTPUT FOR FIRST 2 FILES

FILE: LIPITOR.521.txt
INPUT TEXT (497 chars):
Severe numbness in hand such that it wakes me out of my sleep.
Tingling in feet, soreness in calf muscles.
Occassional sharp shock like pain in head.
I have taken myself off because I can't stand the side-effects.
Never really had high cholesterol but current medical thinking is that lipitor may rev...

---------------------------------------------------------------------

LLM BIO OUTPUT (1017 chars):
Here is the tagged text:

Severe/B-ADR numbness/B-ADR in/I-ADR hand/I-AD such/I-AD that/I-AD it/I-AD wakes/I-AD me/O out/I-AD of/I-AD my/I-AD sleep/O
Tingling/B-ADR in/I-ADR feet/I-AD soreness/B-ADR in/I-ADR calf/I-AD muscles/I-AD
Occassional/B-ADR sharp/B-ADR shock/B-ADR like/I-ADR pain/B-ADR in/I-AD head/I-AD
I/O have/I- taken/I- myself/O off/I- because/I- can't/I- stand/I- the/I- side-effects/B-ADR
Never/I- really/I- had/I- high/O cholesterol/O but/I- current/I- medical/I- thin

Processing:   0%|          | 0/48 [00:00<?, ?it/s]

LIPITOR.579.txt                | P:0.100 R:0.111 F1:0.105
LIPITOR.93.txt                 | P:0.167 R:0.200 F1:0.182
LIPITOR.473.txt                | P:0.250 R:0.200 F1:0.222
LIPITOR.726.txt                | P:0.000 R:0.000 F1:0.000
LIPITOR.864.txt                | P:0.000 R:0.000 F1:0.000
LIPITOR.650.txt                | P:0.105 R:0.286 F1:0.154
LIPITOR.428.txt                | P:0.200 R:0.143 F1:0.167
LIPITOR.128.txt                | P:0.000 R:0.000 F1:0.000
LIPITOR.675.txt                | P:0.273 R:0.500 F1:0.353
LIPITOR.493.txt                | P:0.250 R:0.111 F1:0.154
LIPITOR.39.txt                 | P:0.500 R:0.571 F1:0.533
LIPITOR.267.txt                | P:1.000 R:1.000 F1:1.000
ARTHROTEC.63.txt               | P:0.400 R:0.429 F1:0.414
LIPITOR.635.txt                | P:1.000 R:0.600 F1:0.750
LIPITOR.138.txt                | P:0.333 R:0.500 F1:0.400
LIPITOR.872.txt                | P:0.111 R:0.200 F1:0.143
ARTHROTEC.2.txt                | P:1.000 R:0.167 F1:0.286
LIPITOR.909.tx

In [33]:
# Cell 11: Task 6 - Create SCT Datastore
def create_sct_datastore(ann_filename: str):
    """
    Combine original and SCT annotations into structured datastore
    """
    sct_filepath = os.path.join(SCT_DIR, ann_filename)
    original_filepath = os.path.join(ORIGINAL_DIR, ann_filename)

    # Map text to label from original file
    text_to_label = {}
    line_regex_orig = re.compile(r'^(T\d+)\t(\w+)[\s\d;]+\t(.+)$')

    with open(original_filepath, 'r', encoding='utf-8') as f:
        for line in f:
            match = line_regex_orig.match(line.strip())
            if match:
                text_to_label[match.group(3).strip()] = match.group(2)

    # Parse SCT file
    sct_data = []
    line_regex_sct = re.compile(r'^(TT\d+)\t(.*?)\t(.*?)$')

    with open(sct_filepath, 'r', encoding='utf-8') as f:
        for line in f:
            match = line_regex_sct.match(line.strip())
            if match:
                _, codes_and_descs_raw, text_and_spans = match.groups()

                # Extract ground truth text (after span numbers)
                ground_truth_text = re.split(r'\d+\s\d+', text_and_spans)[-1].strip()
                label = text_to_label.get(ground_truth_text, 'Unknown')

                # Parse SNOMED codes and descriptions
                codes_and_descs = re.findall(r'(\d+)\s*\|\s*(.*?)\s*(?=\||$)', codes_and_descs_raw)

                for code, desc in codes_and_descs:
                    sct_data.append({
                        'sct_code': code.strip(),
                        'sct_description': desc.strip(),
                        'label_type': label,
                        'ground_truth_text': ground_truth_text
                    })

    return pd.DataFrame(sct_data)

# Create datastore
sample_ann = sample_filename.replace('.txt', '.ann')
sct_df = create_sct_datastore(sample_ann)

print("=== SCT DATASTORE (first 10 rows) ===")
print(sct_df.head(10))
print(f"\nTotal entries: {len(sct_df)}")
print(f"Unique codes: {sct_df['sct_code'].nunique()}")

=== SCT DATASTORE (first 10 rows) ===
           sct_code                       sct_description label_type  \
0         271782001                                Drowsy        ADR   
1         246636008                 Blurred vision - hazy        ADR   
2         162076009  Excessive upper gastrointestinal gas        ADR   
3  3384011000036100                             Arthrotec       Drug   
4           3723001                             Arthritis    Disease   
5         102498003                                 Agony    Symptom   
6          76948002                           Severe pain    Symptom   
7          22253000                                  Pain    Symptom   
8         367391008                               Malaise        ADR   

       ground_truth_text  
0             bit drowsy  
1  little blurred vision  
2       gastric problems  
3              Arthrotec  
4              arthritis  
5                  agony  
6                  agony  
7                  pains 